**Описание данных**:

Таблица 1. Даты (dates)

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | уникальный идентификатор компании                 |
| `date_joined` | дата, когда компания стала единорогом  |
| `year_founded` | год основания компании       |


Таблица 2. Финансирование (funding)

| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | уникальный идентификатор компании                 |
| `valuation`        | стоимость компании в долларах США                 |
| `funding`          | cумма привлеченного финансирования в долларах США  |
| `select_investors` | cписок ключевых инвесторов компании     |

Таблица 3. Отрасли промышленности (industries)

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | уникальный идентификатор компании                   |
| `industry`     | отрасль, в которой работает компания   |

Таблица 4. Компании (companies)

| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | уникальный идентификатор компании                      |
| `company`      | название компании                         |
| `city`         | город, в котором находится штаб-квартира компании        |
| `country`      | страна, в которой находится штаб-квартира компании   |
| `continent`    | континент, на котором находится штаб-квартира компании |

**План решения**:

* Найдем отрасли по количеству фирм единорогов за три года

* Выведем таблицу содержащую отрасль, год, количество компаний и среднюю оценку

* Используя CTE отобразим наиболее эффективные отрасли за три года

**Инструменты**: CTE, INNER JOIN, GROUP BY

In [ ]:
SELECT industry, COUNT(*) as num_per_industry
FROM dates
INNER JOIN industries
USING (company_id)
WHERE EXTRACT(year FROM date_joined) in ('2019', '2020', '2021')
GROUP BY industry
ORDER BY num_per_industry DESC
LIMIT 3;

,industry,num_per_industry
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


In [ ]:
SELECT COUNT(*) AS num_unicorns,
        industry,
        EXTRACT(year FROM date_joined) AS year,
        AVG(valuation) AS average_valuation
FROM industries
INNER JOIN dates
USING (company_id)
INNER JOIN funding
USING (company_id)
GROUP BY industry, year

,num_unicorns,industry,year,average_valuation
0,5,Mobile & telecommunications,2017,2.800000e+09
1,4,Internet software & services,2015,1.250000e+09
2,10,Fintech,2018,8.600000e+09
3,4,Mobile & telecommunications,2019,2.000000e+09
4,1,Artificial intelligence,2012,2.000000e+09
...,...,...,...,...
119,7,E-commerce & direct-to-consumer,2022,1.571429e+09
120,2,Other,2015,1.000000e+09
121,4,Consumer & retail,2017,1.050000e+10
122,20,Fintech,2019,6.800000e+09


In [ ]:
WITH top_industries AS
(SELECT industry, COUNT(*)
 FROM industries AS i
 INNER JOIN dates AS d
 USING (company_id)
 WHERE EXTRACT(year FROM date_joined) in ('2019', '2020', '2021')
 GROUP BY industry
 ORDER BY count DESC
 LIMIT 3),

yearly_rankings AS
(SELECT COUNT(*) AS num_unicorns, industry,
        EXTRACT(year FROM date_joined) AS year,
        AVG(valuation) AS average_valuation
    FROM industries AS i
    INNER JOIN dates AS d
    USING (company_id)
    INNER JOIN funding AS f
    USING (company_id)
    GROUP BY industry, year)

SELECT industry, year,
    num_unicorns,
    ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
FROM yearly_rankings
WHERE year in ('2019', '2020', '2021') AND industry in (SELECT industry
                                                        FROM top_industries)
GROUP BY industry, num_unicorns, year
ORDER BY year DESC, num_unicorns DESC

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58
